In [ ]:
import os
import qp
import jax
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import tables_io
from jax import numpy as jnp

from rail.core.data import TableHandle
from rail.core.stage import RailStage
from rail.utils.path_utils import RAILDIR
from rail.core.common_params import SHARED_PARAMS

from rail.shire import ShireInformer, ShireEstimator, hist_outliers, plot_zp_zs_ensemble

jax.config.update("jax_enable_x64", True)

## Select and load data into the datastore

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [ ]:
trainFile = os.path.abspath(os.path.join('.', 'data', 'train_magszgalaxies_lsstroman_gold_hp10552_10k.h5')) #os.path.abspath(os.path.join('./data', 'test_magszgalaxies_lsstroman_gold_hp10552_50k.h5')) #os.path.join(RAILDIR, 'rail/examples_data/testdata/test_dc2_training_9816.hdf5')
testFile = os.path.abspath(os.path.join('.', 'data', 'test_magszgalaxies_lsstroman_gold_hp10552_50k.h5')) #os.path.abspath(os.path.join('../..', 'magszgalaxies_lsstroman_gold_hp10552.h5')) #os.path.join(RAILDIR, 'rail/examples_data/testdata/test_dc2_validation_9816.hdf5')

training_data = DS.read_file("training_data", TableHandle, trainFile)
test_data = DS.read_file("test_data", TableHandle, testFile)

In [ ]:
lsst_filts_dict = {f"{_n}_lsst": "filt_lsst" for _n in "ugrizy"}
roman_filts_dict = {
    "roman_wfi_f106": "",
    "roman_wfi_f129": "",
    "roman_wfi_f158": "",
    "roman_wfi_f184": ""
}

_bands = [ f"mag_{_k}" for _k in {**lsst_filts_dict, **roman_filts_dict} ]
_errbands = [ f"mag_err_{_k}" for _k in {**lsst_filts_dict, **roman_filts_dict} ]
_maglims = {**SHARED_PARAMS['mag_limits'], **{_band: 28 for _band in _bands if 'roman' in _band}}
print(_maglims)

## Inform the estimator, i.e. select a subset of galaxies as templates

In [ ]:
default_dict_inform = dict(
    hdf5_groupname="photometry",
    data_path="./data",
    bands=_bands,
    err_bands=_errbands,
    mag_limits=_maglims,
    spectra_file="dsps_valid_fits_F2SM3_GG_DESI.h5",
    ssp_file="ssp_data_fsps_v3.2_lgmet_age.h5",
    filter_dict={**lsst_filts_dict, **roman_filts_dict},
    wlmin=500.,
    wlmax=25000.,
    dwl=5.,
    colrsbins=40
)

### Prepare two informers: one for each method 'SPS' or 'Legacy'
- 'SPS' recomputes an SED at every redshift based on the star-formation history of the template galaxy before synthesizing the colours for every value of $z$ along the grid
- 'Legacy' computes the SED once at the template galaxy's redshift and uses it to synthesize colours at all $z$ values with the usual transformation $\lambda_\mathrm{obs} = (1+z)\lambda_\mathrm{em}$

In [ ]:
run_shire_inform_sps = ShireInformer.make_stage(
    name="shireSPS_inform_romanlsstSimhp10552_demo",
    output="shireSPS_templates_romanlsstSimhp10552_demo.hf5",
    **default_dict_inform,
    templ_type="SPS"
)

run_shire_inform_legacy = ShireInformer.make_stage(
    name="shireLEG_inform_romanlsstSimhp10552_demo",
    output="shireLEG_templates_romanlsstSimhp10552_demo.hf5",
    **default_dict_inform,
    templ_type="Legacy"
)

### Inform the 'SPS' templates

In [ ]:
%%time
run_shire_inform_sps.inform(training_data)

In [ ]:
templ = run_shire_inform_sps.get_handle("templates")
templ.read()

In [ ]:
run_shire_inform_sps.plot_colrs_templates()

In [ ]:
run_shire_inform_sps.plot_sfh_templates()

In [ ]:
run_shire_inform_sps.hist_colrs_templates()

In [ ]:
run_shire_inform_sps.plot_bpt_templates()

In [ ]:
all_templs_df_sps = run_shire_inform_sps._nuvk_classif()
f, a = plt.subplots(1,1)
sns.scatterplot(
    data=all_templs_df_sps, x="g_lsst-r_lsst", y="r_lsst-i_lsst",
    hue="CAT_NUVK", size='z_p', sizes=(10, 100), alpha=0.5,
    ax=a
)
a.grid()

In [ ]:
run_shire_inform_sps.plot_templ_seds()

In [ ]:
run_shire_inform_sps.plot_line_sed(7, redshift=0.46)

### Inform the 'Legacy' templates

In [ ]:
%%time
run_shire_inform_legacy.inform(training_data)

In [ ]:
run_shire_inform_legacy.plot_templ_seds()

In [ ]:
run_shire_inform_legacy.plot_colrs_templates()

In [ ]:
all_templs_df_leg = run_shire_inform_legacy._nuvk_classif()
f, a = plt.subplots(1,1)
sns.scatterplot(
    data=all_templs_df_leg, x="g_lsst-r_lsst", y="r_lsst-i_lsst",
    hue="CAT_NUVK", size='z_p', sizes=(10, 100), alpha=0.5
)
a.grid()

## Run the photometric redshifts estimation

### Build two estimators
Again, one is for the "SPS" method and the other is for the "Legacy" method. Though both `estimate` stages could work with templates from either `inform` stage, it makes more sense to keep things consistent and load the appropriate `handles` from the corresponding `inform` stage.

In [ ]:
use_prior = True
_suffix = "" if use_prior else "_noprior"

In [ ]:
default_dict_estimate = dict(
    hdf5_groupname="photometry",
    data_path="./data",
    bands=_bands,
    err_bands=_errbands,
    zmin=0.01,
    zmax=3.1,
    nzbins=310,
    mag_limits=_maglims,
    ssp_file="ssp_data_fsps_v3.2_lgmet_age.h5",
    filter_dict={**lsst_filts_dict, **roman_filts_dict},
    wlmin=500.,
    wlmax=25000.,
    dwl=5.,
    no_prior=True,
    chunk_size=5000
)

run_shire_estimate_sps = ShireEstimator.make_stage(
    name="shireSPS_estimate_romanlsstSimhp10552_demo"+_suffix,
    output=f"shireSPS_results_romanlsstSimhp10552_demo{_suffix}.hdf5",
    **default_dict_estimate,
    templ_type="SPS",
    templates=run_shire_inform_sps.get_handle("templates"),
    model=run_shire_inform_sps.get_handle("model")
)

run_shire_estimate_legacy = ShireEstimator.make_stage(
    name="shireLEG_estimate_romanlsstSimhp10552_demo"+_suffix,
    output=f"shireLEG_results_romanlsstSimhp10552_demo{_suffix}.hdf5",
    **default_dict_estimate,
    templ_type="Legacy",
    templates=run_shire_inform_legacy.get_handle("templates"),
    model=run_shire_inform_legacy.get_handle("model")
)

### Run a reference estimator

In [ ]:
typefile = 'training_types_lsstroman_legacy.hdf5'
typ_df = pd.DataFrame()
traintypes = np.array(run_shire_inform_legacy.besttypes)
typ_df['types'] = traintypes #np.where(traintypes > 1, traintypes-1, traintypes)
tables_io.write(typ_df, typefile)
typ_df['types']

In [ ]:
from rail.estimation.algos.bpz_lite import BPZliteInformer, BPZliteEstimator

from rail.core.data import ModelHandle

RAILDIR = "/global/u2/j/jcheval/rail_base/src"

cosmospriorfile = os.path.join(RAILDIR, "rail/examples_data/estimation_data/data/COSMOS31_HDFN_prior.pkl")
cosmosprior = DS.read_file("cosmos_prior", ModelHandle, cosmospriorfile)
sedfile = "COSMOS_seds.list" #os.path.join(RAILDIR, "rail/examples_data/estimation_data/data/SED/COSMOS_seds.list")

bands = ["u", "g", "r", "i", "z", "y"]
lsst_bands = []
lsst_errs = []
lsst_filts = []
for band in bands:
    lsst_bands.append(f"mag_{band}_lsst")
    lsst_errs.append(f"mag_err_{band}_lsst")
    lsst_filts.append(f"DC2LSST_{band}")

robands = ["wfi_f106", "wfi_f129", "wfi_f158", "wfi_f184"]
roman_bands = [f"mag_{band}_roman" for band in robands]
roman_errs = [f"mag_err_{band}_roman" for band in robands]
roman_filts = [f"roman_{band}" for band in robands]

cosmos_dict = dict(
    hdf5_groupname="photometry",
    output=f"BPZ_results_romanlsstSimhp10552_demo{_suffix}.hdf5",
    spectra_file=sedfile,
    bands=_bands,
    err_bands=_errbands,
    filter_list=lsst_filts+roman_filts,
    mag_limits=_maglims,
    zp_errors=np.full(len(lsst_bands+roman_bands), 0.01),
    wlmin=500.,
    wlmax=25000.,
    dwl=5.,
    zmin=0.01,
    zmax=3.1,
    nzbins=310,
    prior_band="mag_i_lsst",
    data_path="/global/u2/j/jcheval/rail_base/src/rail/examples_data/estimation_data/data",
    no_prior=not(use_prior),
    chunk_size=5000
)

inform_bpz = BPZliteInformer.make_stage(
    name="BPZ_inform_romanlsstSimhp10552_demo",
    hdf5_groupname="photometry",
    nondetect_val=jnp.nan,
    bands=_bands,
    err_bands=_errbands,
    filter_list=lsst_filts+roman_filts,
    mag_limits=_maglims,
    zp_errors=np.full(len(lsst_bands+roman_bands), 0.01),
    prior_band="mag_i_lsst",
    wlmin=500.,
    wlmax=25000.,
    dwl=5.,
    data_path="/global/u2/j/jcheval/rail_base/src/rail/examples_data/estimation_data/data",
    nt_array=[10, 5, 16],
    type_file=typefile
)

In [ ]:
%%time
inform_bpz.inform(training_data)

In [ ]:
estimate_bpz = BPZliteEstimator.make_stage(
    name="BPZ_estimate_romanlsstSimhp10552_demo"+_suffix,
    model= cosmosprior, # inform_bpz.get_handle("model"), #
    **cosmos_dict
)

In [ ]:
%%time
estimate_bpz.estimate(test_data)

In [ ]:
pdfs_file_bpz = f"BPZ_results_romanlsstSimhp10552_demo{_suffix}.hdf5"
custom_res_bpz = qp.read(pdfs_file_bpz)
sz = jnp.array(test_data()['photometry']['redshift'])

In [ ]:
a = plot_zp_zs_ensemble(
    custom_res_bpz, sz,
    z_grid=None, key_estim="zmode",
    label='_'.join(['BPZ']+(os.path.splitext(pdfs_file_bpz)[0]).split('_')[2:]))
plt.show()

### Run the "SPS" estimation

In [ ]:
%%time
run_shire_estimate_sps.estimate(test_data)

In [ ]:
pdfs_file_sps = f"shireSPS_results_romanlsstSimhp10552_demo{_suffix}.hdf5"
custom_res_sps = qp.read(pdfs_file_sps)
sz = jnp.array(test_data()['photometry']['redshift'])

In [ ]:
a = plot_zp_zs_ensemble(
    custom_res_sps, sz,
    z_grid=None, key_estim="zmode",
    label='_'.join(['SHIRE_SPS']+(os.path.splitext(pdfs_file_sps)[0]).split('_')[2:]))
plt.show()

### Run the "Legacy" estimation

In [ ]:
%%time
run_shire_estimate_legacy.estimate(test_data)

In [ ]:
pdfs_file_legacy = f"shireLEG_results_romanlsstSimhp10552_demo{_suffix}.hdf5"
custom_res_legacy = qp.read(pdfs_file_legacy)
# sz = jnp.array(test_data()['photometry']['redshift']) -- Unnecessary

In [ ]:
a = plot_zp_zs_ensemble(
    custom_res_legacy, sz,
    z_grid=None, key_estim="zmode",
    label='_'.join(['SHIRE_Legacy']+(os.path.splitext(pdfs_file_legacy)[0]).split('_')[2:]))
plt.show()

### Compare outliers distribution between both methods

In [ ]:
hist_outliers(
    custom_res_sps, sz, label1='_'.join(['SHIRE_SPS']+(os.path.splitext(pdfs_file_sps)[0]).split('_')[2:]),
    qp_ens_2=custom_res_legacy, label2='_'.join(['SHIRE_Legacy']+(os.path.splitext(pdfs_file_legacy)[0]).split('_')[2:]),
    qp_ens_3=custom_res_bpz, label3='_'.join(['BPZ']+(os.path.splitext(pdfs_file_bpz)[0]).split('_')[2:])
)
plt.show()

## Evaluate posteriors using `RAIL` 

Check out [Evaluation_demo_LSSTsim.ipynb](Evaluation_demo_LSSTsim.ipynb) !

## Build a pipeline with `ceci`

In [ ]:
import ceci
pipe = ceci.Pipeline.interactive()
stages = [run_shire_inform_sps, run_shire_estimate_sps]
for stage in stages:
    pipe.add_stage(stage)
pipe.stage_execution_config['shireSPS_estimate_romanlsstSimhp10552_demo'+_suffix].nprocess=1

In [ ]:
pipe.initialize(
    dict(
        training_data=trainFile,
        test_data=testFile
    ),
    dict(
        output_dir='.',
        log_dir='.',
        resume=False
    ),
    None
)

In [ ]:
pipe.save(f'rail_shireSPS_romanlsstsim_pz_demo{_suffix}.yml')

In [ ]:
pr = ceci.Pipeline.read(f'rail_shireSPS_romanlsstsim_pz_demo{_suffix}.yml')

In [ ]:
pr.run()